## 1. Import libraries and dependencies

In [1]:
from src.preprocess.preprocess import Preprocess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

2024-05-19 16:15:03.780413: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-19 16:15:03.783030: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-19 16:15:03.818070: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 16:15:04.475852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/home/hubert/Workspace/Python/ML Project/languages
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl'}
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs/validated.tsv', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt/validated.tsv', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl/validated.tsv'}
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs/clip_durations.tsv', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt/clip_durations.tsv', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl/clip_durations.tsv'}


## 2. Preprocess audio

In [2]:
ORIGIN_SAMPLE_RATE = 48_000
FINAL_SAMPLE_RATE = 16_000
MAX_CLIENT_ID_AMOUNT = 1000
MIN_CLIP_DURATION_MS = 4000
SET_SIZE = 200
BATCH_SIZE = 32

In [3]:
Preprocess.set_origin_sample_rate(ORIGIN_SAMPLE_RATE)
Preprocess.set_final_sample_rate(FINAL_SAMPLE_RATE)
Preprocess.set_max_client_id_amount(MAX_CLIENT_ID_AMOUNT)
Preprocess.set_min_clip_duration_ms(MIN_CLIP_DURATION_MS)
Preprocess.set_set_size(SET_SIZE)
Preprocess.set_batch_size(BATCH_SIZE)

<class 'int'>


In [4]:
print(f"Original sample rate: {Preprocess.ORIGIN_SAMPLE_RATE}")
print(f"Final sample rate: {Preprocess.SAMPLE_RATE}")
print(f"Max client id amount: {Preprocess.MAX_CLIENT_ID_AMOUNT}")
print(f"Min clip duration in milliseconds: {Preprocess.MIN_CLIP_DURATION_MS}")
print(f"Set size: {Preprocess.SET_SIZE}")
print(f"One language set size: {Preprocess.ONE_LANG_SET_SIZE}")

print("-"*25 + "SETS SIZES" + "-"*25)
print(f"Training set size: {Preprocess.SET_HPARAMS.train_size}")
print(f"Val set size: {Preprocess.SET_HPARAMS.val_size}")
print(f"Test set size: {Preprocess.SET_HPARAMS.test_size}")
print(f"Is test + val + train is equal one lang set size: "
      f"\n{Preprocess.SET_HPARAMS.test_size + Preprocess.SET_HPARAMS.val_size + Preprocess.SET_HPARAMS.train_size} == {Preprocess.ONE_LANG_SET_SIZE}")

Original sample rate: 48000
Final sample rate: 16000
Max client id amount: 1000
Min clip duration in milliseconds: 4000
Set size: 200
One language set size: 66
-------------------------SETS SIZES-------------------------
Training set size: 39
Val set size: 13
Test set size: 14
Is test + val + train is equal one lang set size: 
66 == 66


In [5]:
train_dataset, val_dataset, test_dataset = Preprocess().preprocess()

PROCESSED LANGUAGES: cs


2024-05-19 16:15:05.572992: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


---AUDIO META INFO DONE cs---
---SET SPLIT DONE 0 cs---


2024-05-19 16:15:06.394159: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA


---DATASET DONE 0 cs---
---SET SPLIT DONE 1 cs---


...

---AUDIO META INFO DONE pl---
---SET SPLIT DONE 0 pl---
---DATASET DONE 0 pl---
---SET SPLIT DONE 1 pl---


2024-05-19 16:15:12.787727: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


---DATASET DONE 1 pl---


2024-05-19 16:15:13.805415: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


SHUFFLE
BATCHING


In [6]:
for audio, label in train_dataset.take(120):
    print(audio.shape)

(32, 1991, 257, 1)
(32, 1991, 257, 1)
(32, 1991, 257, 1)


2024-05-19 16:15:14.571470: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
for audio, label in val_dataset.take(112):
    print(audio.shape)

(32, 1991, 257, 1)


2024-05-19 16:15:15.065880: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
for audio, label in test_dataset.take(112):
    print(audio.shape)

(32, 1991, 257, 1)


2024-05-19 16:15:15.190061: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
counter = 0
for batch, label in train_dataset:
        print(batch.shape)
        for sample in batch:
                print(sample)
        counter += 1
print(counter)


(32, 1991, 257, 1)
tf.Tensor(
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]], shape=(1991, 257, 1), dtype=float32)
tf.Tensor(
[[[3.2255962e-03]
  [1.7887012e-03]
  [6.0506567e-04]
  ...
  [4.1617277e-06]
  [6.3522600e-07]
  [7.9534948e-07]]

 [[3.8727527e-03]
  [2.7184419e-03]
  [1.8319676e-03]
  ...
  [3.8089868e-06]
  [1.6218230e-06]
  [7.0047099e-07]]

 [[1.5422439e-03]
  [3.8320678e-03]
  [5.1037963e-03]
  ...
  [4.7813319e-06]
  [1.7406787e-06]
  [2.2468157e-08]]

 ...

 [[1.3408210e-04]
  [1.1932923e-03]
  [1.1181656e-03]
  ...
  [1.9901204e-06]
  [1.3644967e-07]
  [7.2235707e-08]]

 [[3.8349474e-04]
  [8.7170780e-04]
  [6.9986162e-04]
  ...
  [2.7840867e-06]
  [8.1404050e-07]
  [9.1924448e-07]]

 [[1.5661558e-03]
  [7.448124

2024-05-19 16:15:15.497652: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
counter = 0
for batch, label in val_dataset:
        print(tf.rank(batch))
        for sample in batch:
                print(tf.rank(sample))
        counter += 1
print(counter)

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype

2024-05-19 16:15:15.630133: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
counter = 0
for batch, label in test_dataset:
        # print(tf.shape(batch))
        for sample in batch:
                print(tf.shape(sample))
        counter += 1
print(counter)

tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=int32)
tf.Tensor([1991  257    1], shape=(3,), dtype=

2024-05-19 16:15:15.772357: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
train_dataset.as_numpy_iterator().next()

(array([[[[3.22559616e-03],
          [1.78870116e-03],
          [6.05065667e-04],
          ...,
          [4.16172770e-06],
          [6.35226002e-07],
          [7.95349479e-07]],
 
         [[3.87275266e-03],
          [2.71844189e-03],
          [1.83196762e-03],
          ...,
          [3.80898678e-06],
          [1.62182300e-06],
          [7.00470991e-07]],
 
         [[1.54224387e-03],
          [3.83206783e-03],
          [5.10379625e-03],
          ...,
          [4.78133188e-06],
          [1.74067873e-06],
          [2.24681571e-08]],
 
         ...,
 
         [[1.34082104e-04],
          [1.19329232e-03],
          [1.11816556e-03],
          ...,
          [1.99012038e-06],
          [1.36449671e-07],
          [7.22357072e-08]],
 
         [[3.83494742e-04],
          [8.71707802e-04],
          [6.99861615e-04],
          ...,
          [2.78408675e-06],
          [8.14040504e-07],
          [9.19244485e-07]],
 
         [[1.56615581e-03],
          [7.44812423e-04]

In [13]:
samples, labels = train_dataset.as_numpy_iterator().next()
print()
print(samples.shape, labels.shape)
samples, labels = val_dataset.as_numpy_iterator().next()
print(samples.shape, labels.shape)
samples, labels = test_dataset.as_numpy_iterator().next()
print(samples.shape, labels.shape)


(32, 1991, 257, 1) (32, 3)
(32, 1991, 257, 1) (32, 3)
(32, 1991, 257, 1) (32, 3)


In [14]:
samples, labels = train_dataset.as_numpy_iterator().next()

In [15]:
samples.shape

(32, 1991, 257, 1)

In [22]:
input_shape = samples.shape[1:]
input_shape

(1991, 257, 1)

In [16]:
test_dataset.element_spec

(TensorSpec(shape=(32, 1991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 3), dtype=tf.float32, name=None))

In [17]:
val_dataset.element_spec

(TensorSpec(shape=(32, 1991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 3), dtype=tf.float32, name=None))

In [18]:
train_dataset.element_spec

(TensorSpec(shape=(32, 1991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 3), dtype=tf.float32, name=None))

## 4. Create model

In [43]:
model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 1991, 257, 32)  │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 995, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 995, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 497, 64, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 497, 64, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2035712)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │   130,285,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130,341,571 (497.21 MB)

 Trainable params: 130,341,571 (497.21 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
train_dataset.element_spec

(TensorSpec(shape=(32, 1991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 3), dtype=tf.float32, name=None))

In [39]:
val_dataset.element_spec

(TensorSpec(shape=(32, 1991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 3), dtype=tf.float32, name=None))

In [40]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, batch_size=BATCH_SIZE)

Epoch 1/10


2024-05-19 16:23:25.361093: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2095869952 exceeds 10% of free system memory.
2024-05-19 16:23:26.394902: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1043333120 exceeds 10% of free system memory.
2024-05-19 16:23:30.349273: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1043333120 exceeds 10% of free system memory.
2024-05-19 16:23:32.000114: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1043333120 exceeds 10% of free system memory.
2024-05-19 16:23:32.000169: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2095869952 exceeds 10% of free system memory.


3/3 ━━━━━━━━━━━━━━━━━━━━ 57s 16s/step - accuracy: 0.2253 - loss: 114.8294 - val_accuracy: 0.3750 - val_loss: 24.4098
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 27s 9s/step - accuracy: 0.3893 - loss: 6.1676 - val_accuracy: 0.4375 - val_loss: 1.3365
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 8s/step - accuracy: 0.4154 - loss: 1.2093 - val_accuracy: 0.4688 - val_loss: 0.9669
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 8s/step - accuracy: 0.6068 - loss: 0.9261 - val_accuracy: 0.5312 - val_loss: 0.9642
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 8s/step - accuracy: 0.8398 - loss: 0.8560 - val_accuracy: 0.5312 - val_loss: 0.8370
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 8s/step - accuracy: 0.9609 - loss: 0.4839 - val_accuracy: 0.5000 - val_loss: 0.9579
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 7s/step - accuracy: 0.9948 - loss: 0.3625 - val_accuracy: 0.3750 - val_loss: 1.1526
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step - accuracy: 0.9688 - loss: 0.2364 - val_accuracy: 0.4688 - val_loss: 1.2698
Epoch 9/10
3/3 

KeyboardInterrupt: 

In [41]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f'\nTest accuracy: {test_acc}')

1/1 - 2s - 2s/step - accuracy: 0.4062 - loss: 1.2568

Test accuracy: 0.40625


In [42]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()


NameError: name 'history' is not defined